In [1]:
!bash /home/azureuser/cloudfiles/code/blobfuse/blobfuse_raadsinformatie.sh

In [2]:
import sys
sys.path.append("..")

# Select where to run notebook: "azure" or "local"
my_run = "azure"

import my_secrets as sc
import settings as st

if my_run == "azure":
    import config_azure as cf
elif my_run == "local":
    import config as cf

### Goal: remove faulty documents
The goal is to remove faulty documents where mistakes were made when converting pdf into txt.

In [3]:
# load data
import ast
import pandas as pd

txtfiles = pd.read_pickle(f"{cf.output_path}/txtfiles.pkl")
# txtfiles['tokens'] = txtfiles['tokens'].apply(ast.literal_eval)
display(txtfiles.head())
df = txtfiles.copy()
df['clean_text'] = df['clean_tokens'].apply(lambda x: ' '.join(x))


,label,path,id,set,text,tokens,token_count,clean_tokens,clean_tokens_count,pdf_path,num_pages
0,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,0,train,Gemeente Amsterdam\n% Gemeenteraad R\n% Gemeen...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",395,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",205,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0
1,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,1,train,Gemeente Amsterdam\n\n% Gemeenteraad R\n\n% Ge...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",390,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",197,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0
2,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,2,train,Gemeente Amsterdam\n\n% Gemeenteraad R\n\n% Ge...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",389,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",192,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0
3,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,3,train,Gemeente Amsterdam\n\n% Gemeenteraad R\n\n% Ge...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",464,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",225,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0
4,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,4,test,x Gemeente Amsterdam R\nGemeenteraad\n\n% Geme...,"[x, Gemeente, Amsterdam, R, Gemeenteraad, %, G...",261,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",134,/home/azureuser/cloudfiles/code/blobfuse/raads...,1.0


In [4]:
display(df['clean_tokens_count'].describe())
class_describe = df.groupby('label')['clean_tokens_count'].describe()
display(class_describe)

count     33117.000000
mean       1392.480448
std        7326.313739
min           3.000000
25%         152.000000
50%         278.000000
75%         622.000000
max      143782.000000
Name: clean_tokens_count, dtype: float64

,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
Actualiteit,996.0,360.197791,1833.846209,39.0,124.00,175.0,311.25,55400.0
Adviesaanvraag,2442.0,825.560197,1517.594501,33.0,287.00,486.0,818.00,39865.0
Agenda,3537.0,559.212327,2187.277067,17.0,180.00,337.0,515.00,72471.0
Amendement,1969.0,1274.853225,3332.397979,15.0,118.00,179.0,595.00,29968.0
Begroting,1967.0,7290.902898,26004.241417,24.0,125.00,203.0,1179.00,137501.0
Besluit,775.0,513.227097,889.155749,50.0,95.50,230.0,657.00,13651.0
Brief,1995.0,881.689724,954.436225,3.0,369.00,636.0,1134.00,21285.0
Factsheets,234.0,3347.893162,9863.108583,60.0,634.75,1527.0,2937.00,131816.0
Motie,8336.0,264.074976,584.938299,60.0,120.00,149.0,204.00,21285.0


##### Remove image gibberish
images results in loads of gibberish (ususally tokens of 1,2,3 or 4 characters long). Check if tokens with less than 5 characters are a  dutch words, else remove the token. 


In [5]:
import nltk
nltk.download('words')

from nltk.corpus import words
dutch_words = set(words.words('nl'))

def half_short_tokens(tokens):
    num_short_tokens = sum(1 for token in tokens if len(token) <= 2)
    return num_short_tokens >= len(tokens) / 2

df['image'] = df['clean_tokens'].apply(half_short_tokens)

def remove_gibberish(tokens):
    clean_tokens = []
    for token in tokens:
        if token.lower() in dutch_words:
            clean_tokens.append(token)

    return clean_tokens

df['no_gibberish_tokens'] = df['clean_tokens'].apply(remove_gibberish)



KeyboardInterrupt: 

##### Check if text if extracted from image (for example a map)
This text will just be gibberish and we want to filter those docs out. We do this by removing documents of which a half or more of the tokens have a length of 2 or less (so either one or two characters).

In [47]:
def half_short_tokens(tokens):
    num_short_tokens = sum(1 for token in tokens if len(token) <= 2)
    return num_short_tokens >= len(tokens) / 2

df['image'] = df['clean_tokens'].apply(half_short_tokens)

for index, row in df.loc[df['image']==True].iterrows():
    print(row['label'], row['clean_tokens_count'], row['path'])
    print(row['clean_tokens'])
    print('\n\n')
    

display(df.loc[df['image']==True])

Onderzoeksrapport 2626 /home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/OpenResearch/onderzoek-in-de-gemeenteraad/onderzoeken-rapporten-2017-gemeenteraad/nieuw-dashboard-citymarketing-2018-2020.pdf.ocr
['ES', 'mn', 'eg', 'ee', 'Ee', 'ee', 'ren', 'pmen', 'Ei', 'an', 'mn', 'ee', 'Ep', 'Ee', 'ln', 're', 'Es', 'nn', 'Ennn', 'Et', 'ene', 'Tr', 'ee', 'ee', 'Ek', 'pd', 'nele', 'nt', 'ee', 'ee', 'EE', 're', 'nk', 'EE', 'Bel', 'Ee', 'Zld', 'le', 'Nn', 'amsterdam', 'marketing', 'Bree', 'DEE', 'ke', 'ra', 'ALi', 'Eg', 'NE', 'EO', 'Sen', 'Rn', 'kn', 'jm', 'he', 'AA', 'mma', 'Et', 'WP', 'TN', 'PS', 'Bm', 'ne', 'gl', 'nn', 'ef', 'ed', 'Ke', 'ae', 'mn', 'LATE', 'Ep', 'Pe', 'AE', 'EA', 'EEE', 'AE', 'Ten', '46', 'egt', 'OT', 'LS', 'Ae', 'lr', 'RN', 're', 'Red', 'rd', 'ELS', 'an', 'HD', 'Dj', 'ed', 'le', 'hk', 'TA', 'RR', 'ENT', 'ACS', 'ANT', 'ine', 'VE', 'pen', 'TTN', 'MK', 'wi', 'pr', 'Eh', 'ne', '|E', 'AE', 'RT', 'eden', 'ET', 'red', 'Ps', 'RED', 'AE', 'Ni', 'Wer', 'ee', 'Eee', 'Naam', 'EE', 

,label,path,id,set,text,tokens,token_count,clean_tokens,clean_tokens_count,pdf_path,num_pages,clean_text,tokens_page,image
8448,Onderzoeksrapport,/home/azureuser/cloudfiles/code/blobfuse/raads...,8448,train,ES mn eg ee Ee en.\nee ren pmen Ei an mn ee Ep...,"[ES, mn, eg, ee, Ee, en, ., ee, ren, pmen, Ei,...",4676,"[ES, mn, eg, ee, Ee, ee, ren, pmen, Ei, an, mn...",2626,/home/azureuser/cloudfiles/code/blobfuse/raads...,12.0,ES mn eg ee Ee ee ren pmen Ei an mn ee Ep Ee l...,218.833333,True
8773,Onderzoeksrapport,/home/azureuser/cloudfiles/code/blobfuse/raads...,8773,val,‚ f Ae 2 ee Ee „ ad and\n_ eK LR sin Pld De a ...,"[‚, f, Ae, 2, ee, Ee, „, ad, and, _, eK, LR, s...",1520,"[Ae, ee, Ee, ad, and, eK, LR, sin, Pld, pd, ‚4...",869,/home/azureuser/cloudfiles/code/blobfuse/raads...,4.0,Ae ee Ee ad and eK LR sin Pld pd ‚4 arm LEET E...,217.250000,True
8937,Onderzoeksrapport,/home/azureuser/cloudfiles/code/blobfuse/raads...,8937,train,EA oe ale AEN KON\nais eE BEE:\n\ rd Wi kk „in...,"[EA, oe, ale, AEN, KON, ais, eE, BEE, :, \, rd...",13430,"[EA, oe, ale, AEN, ais, eE, BEE, rd, Wi, kk, i...",7378,/home/azureuser/cloudfiles/code/blobfuse/raads...,18.0,EA oe ale AEN ais eE BEE rd Wi kk inr WR ef RE...,409.888889,True
8975,Onderzoeksrapport,/home/azureuser/cloudfiles/code/blobfuse/raads...,8975,train,en ikl i dr 4\n| oi\ner a z \ ' Á 5 e\nee A Fk...,"[en, ikl, i, dr, 4, |, oi, er, a, z, \, ', Á, ...",10344,"[ikl, dr, oi, ee, Fk, kT, RAR, OC, PF, PIN, pf...",6550,/home/azureuser/cloudfiles/code/blobfuse/raads...,23.0,ikl dr oi ee Fk kT RAR OC PF PIN pf ih IR TT A...,284.782609,True
9136,Onderzoeksrapport,/home/azureuser/cloudfiles/code/blobfuse/raads...,9136,train,"OMster Nene end"" WNW MULT-CRANE con eh\ndam ne...","[OMster, Nene, end, '', WNW, MULT-CRANE, con, ...",13431,"[OMster, Nene, end, '', WNW, MULT-CRANE, con, ...",7126,/home/azureuser/cloudfiles/code/blobfuse/raads...,20.0,OMster Nene end '' WNW MULT-CRANE con eh dam n...,356.300000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31521,Schriftelijke Vragen,/home/azureuser/cloudfiles/code/blobfuse/raads...,31531,test,dl \ \ Ik SN ON N\nU In NR | 5 Os\naj \ \ De Ì...,"[dl, \, \, Ik, SN, ON, N, U, In, NR, |, 5, Os,...",603,"[dl, SN, ON, NR, Os, aj, QX, Ss, AR, Le, Fold,...",294,/home/azureuser/cloudfiles/code/blobfuse/raads...,1.0,dl SN ON NR Os aj QX Ss AR Le Fold Se On WÁ= S...,294.000000,True
31666,Schriftelijke Vragen,/home/azureuser/cloudfiles/code/blobfuse/raads...,31676,train,x Gemeente Amsterdam R\nGemeenteraad\n% Gemeen...,"[x, Gemeente, Amsterdam, R, Gemeenteraad, %, G...",2287,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",1346,/home/azureuser/cloudfiles/code/blobfuse/raads...,4.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad S...,336.500000,True
31855,Schriftelijke Vragen,/home/azureuser/cloudfiles/code/blobfuse/raads...,31865,train,"Pe ae 5 8 Á ee\n‚ anne garneer , ne\n: gern a ...","[Pe, ae, 5, 8, Á, ee, ‚, anne, garneer, ,, ne,...",375,"[Pe, ae, ee, anne, garneer, ne, gern, fenraat-...",184,/home/azureuser/cloudfiles/code/blobfuse/raads...,1.0,Pe ae ee anne garneer ne gern fenraat-e tf Bui...,184.000000,True
32597,Schriftelijke Vragen,/home/azureuser/cloudfiles/code/blobfuse/raads...,32607,train,owe aeee EI PE Par ol\nF rn en nn\n& — Dee ee ...,"[owe, aeee, EI, PE, Par, ol, F, rn, en, nn, &,...",992,"[owe, aeee, EI, PE, Par, ol, rn, nn, Dee, ee, ...",382,/home/azureuser/cloudfiles/code/blobfuse/raads...,1.0,owe aeee EI PE Par ol rn nn Dee ee ES eene nn ...,382.000000,True


##### Average tokens per page

In [6]:
df['tokens_page'] = df['clean_tokens_count']/df['num_pages']
display(df['tokens_page'].describe())
class_describe = df.groupby('label')['tokens_page'].describe()
display(class_describe)
display(df.head())

count    33114.000000
mean       146.037663
std         61.650676
min          3.000000
25%        104.750000
50%        135.000000
75%        180.000000
max       1744.500000
Name: tokens_page, dtype: float64

,count,mean,std,min,25%,50%,75%,max
label,,,,,,,,
Actualiteit,996.0,130.756199,40.243522,39.000000,101.000000,127.000000,159.000000,307.000000
Adviesaanvraag,2442.0,184.328488,59.899469,33.000000,142.000000,181.666667,221.226966,794.000000
Agenda,3537.0,113.798911,33.555671,17.000000,96.000000,113.000000,130.500000,498.500000
Amendement,1968.0,145.302770,70.937087,7.500000,100.000000,129.000000,171.000000,758.000000
Begroting,1967.0,143.836542,60.184076,17.500000,103.000000,130.000000,172.200000,930.000000
Besluit,775.0,118.047598,43.077913,33.000000,83.525000,116.000000,147.000000,327.000000
Brief,1995.0,186.369043,42.818155,3.000000,156.333333,191.333333,217.200000,287.375000
Factsheets,234.0,229.006375,198.944347,38.000000,131.304688,193.312500,269.187500,1744.500000
Motie,8334.0,121.479216,38.506947,30.333333,94.000000,115.000000,140.000000,309.250000


,label,path,id,set,text,tokens,token_count,clean_tokens,clean_tokens_count,pdf_path,num_pages,clean_text,tokens_page
0,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,0,train,Gemeente Amsterdam\n% Gemeenteraad R\n% Gemeen...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",395,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",205,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad M...,102.5
1,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,1,train,Gemeente Amsterdam\n\n% Gemeenteraad R\n\n% Ge...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",390,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",197,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad M...,98.5
2,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,2,train,Gemeente Amsterdam\n\n% Gemeenteraad R\n\n% Ge...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",389,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",192,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad M...,96.0
3,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,3,train,Gemeente Amsterdam\n\n% Gemeenteraad R\n\n% Ge...,"[Gemeente, Amsterdam, %, Gemeenteraad, R, %, G...",464,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",225,/home/azureuser/cloudfiles/code/blobfuse/raads...,2.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad M...,112.5
4,Motie,/home/azureuser/cloudfiles/code/blobfuse/raads...,4,test,x Gemeente Amsterdam R\nGemeenteraad\n\n% Geme...,"[x, Gemeente, Amsterdam, R, Gemeenteraad, %, G...",261,"[Gemeente, Amsterdam, Gemeenteraad, Gemeentebl...",134,/home/azureuser/cloudfiles/code/blobfuse/raads...,1.0,Gemeente Amsterdam Gemeenteraad Gemeenteblad M...,134.0


##### Actualiteit
After checking the lowest 1 percent of number of tokens (10 documents which have less than 58 tokens) were checked using the PDFs. These documents are just very short, thus not a mistake of the txt extraction. 

In [7]:
actualiteit = df.loc[df['label']=='Actualiteit']
threshold = actualiteit['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = actualiteit[actualiteit['clean_tokens_count'] < threshold]
# actualiteit['clean_tokens_count'].describe()


##### Adviesaanvraag
Txt extraction from PDFs went well, except for image. The images result in gibberish in the data. Additionally, it does not seem like there are only adviesaanvragen in there. 

In [10]:
adviesaanvraag = df.loc[df['label']=='Adviesaanvraag']
threshold = adviesaanvraag['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = adviesaanvraag[adviesaanvraag['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# adviesaanvraag['clean_tokens_count'].describe()


24


#### Agenda
Agenda's can just be very short. Txt extraction went well.

In [12]:
subdf = df.loc[df['label']=='Agenda']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


#### Amendement
Not only amendement docs are included. txt extraction went well.

In [17]:
subdf = df.loc[df['label']=='Amendement']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


#### Begroting
It does not seem like there are only begrotingen. txt went well. except for tables (file:///C:/Users/femke/Documents/MasterThesis/discardfiles/1543430.pdf)

In [19]:
subdf = df.loc[df['label']=='Begroting']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


#### Besluit
Looks good

In [21]:
subdf = df.loc[df['label']=='Besluit']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


#### Brief
Remove docs with less than 75 tokens. These are some weird poster. The rest looks good.

In [24]:
subdf = df.loc[df['label']=='Brief']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


#### Factsheets
Lots of posters. Lots of images included

In [30]:
subdf = df.loc[df['label']=='Factsheets']
threshold = subdf['clean_tokens_count'].quantile(0.1)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


#### Motie
I cant see the original PDFs. Looks good tho.

In [32]:
subdf = df.loc[df['label']=='Motie']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
print(len(lower_5_percentile_rows))

for index, row in lower_5_percentile_rows.iterrows():
    print(row['clean_tokens_count'], row['path'])
    print(row['text'])
    print('\n\n')

subdf['clean_tokens_count'].describe()


84
60 /home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/raadsinformatie/Moties/2013/11/https:||amsterdam.raadsinformatie.nl|document|4433190|4|952_13.ocr
X Gemeente Amsterdam R
Gemeenteraad
% Gemeenteblad
% Motie
Jaar 2013
Afdeling 1
Nummer 952
Publicatiedatum 15 november 2013
Ingekomen op 6 november 2013
Ingekomen onder 915
Behandeld op 7 november 2013
Status Aangenomen
Onderwerp
Motie van de raadsleden de heer De Goede, mevrouw Ruigrok en mevrouw
Van der Heijden inzake de begroting voor 2014 (uitvoering Wet milieubeheer).
Aan de gemeenteraad
Ondergetekenden hebben de eer voor te stellen:
De raad,
Gehoord de discussie over de begroting voor 2014;
Verzoekt het college van burgemeester en wethouders:
De leden van de gemeenteraad,
M. de Goede
M.H. Ruigrok
LB. van der Heijden
1




60 /home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/raadsinformatie/Moties/2022/1/https:||amsterdam.raadsinformatie.nl|document|11108953|1|065_22_Motie+Yilmaz+uitgebreide+evaluatie+van+het+nieuw

count     8336.000000
mean       264.074976
std        584.938299
min         60.000000
25%        120.000000
50%        149.000000
75%        204.000000
max      21285.000000
Name: clean_tokens_count, dtype: float64

#### Onderzoeksrapport      
Include posters, presentations? (/home/azureuser/cloudfiles/code/blobfuse/raadsinformatie/OpenResearch/onderzoek-in-de-gemeenteraad/onderzoeken-rapporten-2019-gemeenteraad/5g-technische-sessie.pdf.ocr
)


Includes images.

In [36]:
subdf = df.loc[df['label']=='Onderzoeksrapport']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


#### Raadsadres
remove less than 25 tokens. looks good.

In [39]:
subdf = df.loc[df['label']=='Raadsadres']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


#### Schriftelijke Vragen
Cant open PDFs. Looks like a mess. Middle looks much better. Should remove lowest 0.01 cleaned_tokens_count

In [46]:
subdf = df.loc[df['label']== 'Schriftelijke Vragen']
threshold = subdf['clean_tokens_count'].quantile(0.01)
# threshold2 = subdf['clean_tokens_count'].quantile(0.02)

lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# lower_5_percentile_rows = subdf[(subdf['clean_tokens_count'] < threshold2) & (subdf['clean_tokens_count']>threshold)].sort_values(by=['clean_tokens_count'])

# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()



#### Termijnagenda
Does include many tables.

In [48]:
subdf = df.loc[df['label']=='Termijnagenda']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


#### Verslag
Should rename into raadsnotulen. looks good

In [51]:
subdf = df.loc[df['label']=='Verslag']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()


#### Voordracht
Looks good

In [54]:
subdf = df.loc[df['label']=='Voordracht']
threshold = subdf['clean_tokens_count'].quantile(0.01)
lower_5_percentile_rows = subdf[subdf['clean_tokens_count'] < threshold].sort_values(by=['clean_tokens_count'])
# print(len(lower_5_percentile_rows))

# for index, row in lower_5_percentile_rows.iterrows():
#     print(row['clean_tokens_count'], row['path'])
#     print(row['text'])
#     print('\n\n')

# subdf['clean_tokens_count'].describe()
